# FlowDex Incident Triage Walkthrough
This notebook demonstrates how to seed durable memory, issue an inference request, and inspect the response metadata.

In [ ]:
import os
import json
import requests

BASE_URL = os.environ.get('FLOWDEX_BASE', 'http://localhost:8787')
API_KEY = os.environ.get('FLOWDEX_API_KEY')
HEADERS = {'Content-Type': 'application/json'}
if API_KEY:
    HEADERS['X-API-Key'] = API_KEY

memory_payload = {
    'id': 'incident.runbook',
    'title': 'Major Incident Checklist',
    'body': '1) Page responders\n2) Check dashboards\n3) Draft customer update',
    'tags': ['incident', 'customer'],
}
requests.post(f'{BASE_URL}/memory/put', json=memory_payload, headers=HEADERS, timeout=30).raise_for_status()

infer_payload = {
    'task': 'incident_triage',
    'model': 'anthropic/claude-3-5-sonnet',
    'inputs': {
        'user': 'Summarize the last incident and propose next steps.',
        'context': ['incident.runbook'],
        'tool_hints': [],
    },
}
response = requests.post(f'{BASE_URL}/infer', json=infer_payload, headers=HEADERS, timeout=60)
response.raise_for_status()
print(json.dumps(response.json(), indent=2))
